# CAFA 5 Competition : Protein Function Prediction

In [ ]:
from utils_local import *

In [ ]:
model_name="cn" 
TRAIN = True
embeddings_source1="T5"
embeddings_source2="EMS2"

In [ ]:


PATHS = {
    "PATH": PATH, 
    "PATH_LABELS": PATH_LABELS,
    "PATH_DATAFRAMES_2D": PATH_DATAFRAMES_2D,
    "CHECKPOINT": {"local":'../results/models_folds/models_{}', "kaggle": '/kaggle/working/models_{}'},
    "PREDICTIONS": {"local":'../results/preds_cafa_embeddings_bp_2emb/preds_fold{}.tsv', "kaggle": 'preds_fold{}.tsv'},
    "LABELS": {"local":'../results/preds_cafa_embeddings_bp_2emb/labels_used_fold{}.tsv', "kaggle": 'labels_used_fold{}.tsv'},
}

In [ ]:
import pandas as pd

sub = pd.read_csv(f"{PATH}cafa-5-protein-function-prediction/sample_submission.tsv", sep= "\t", header = None)
sub.columns = ["The Protein ID", "The Gene Ontology term (GO) ID", "Predicted link probability that GO appear in Protein"]
sub.head(5)

In [ ]:
from Bio import SeqIO
print("Loading train set ProtBERT Embeddings...")
fasta_train = SeqIO.parse(config.train_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_train)))
fasta_test= SeqIO.parse(config.test_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_test)))

In [ ]:
weights=pd.read_csv(f"{PATH}cafa-5-protein-function-prediction/IA.txt",sep="\t",header=None,index_col=0).rename(columns={1:"weight"})


In [ ]:
 labels = pd.read_csv(config.train_labels_path, sep = "\t")
 labels.shape

In [ ]:
 labels= labels.loc[~labels.term.isin(list(weights[weights.weight==0].index))]
 labels.shape

In [ ]:
# Directories for the different embedding vectors : 
embeds_map = {
    "T5" : "my-t5embeds",
    "ProtBERT" : "protbert-embeddings-for-cafa5",
    "EMS2" : "cafa-5-ems-2-embeddings-numpy"
}

# Length of the different embedding vectors :
embeds_dim = {
    "T5" : 1024,
    "ProtBERT" : 1024,
    "EMS2" : 1280
}

In [ ]:
 LEARNING_RATE=0.001
 WARMUP_STEPS=7
 class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps=1):
        super(CustomSchedule, self).__init__()

        self.initial_lr = tf.cast(initial_lr, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        return tf.math.minimum(self.initial_lr, self.initial_lr * (step/self.warmup_steps))  
    
    def get_config(self):
        return {"initial_learning_rate": self.initial_lr}
    

'''
PredictionFnCallback is used for:
1. Loading validation data
2. FOGModel data preparation
3. Prediction
4. Scoring and save

'''

In [ ]:
def get_dataset( datatype, embeddings_source):
        ids_ = np.load(f"{PATH}" + embeds_map["T5"] + "/" + datatype + "_ids.npy")
       
        if embeddings_source in ["ProtBERT", "EMS2"]:
            embeds = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_embeddings.npy")
            ids = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
            dic_ids={e:i for i, e in enumerate(ids)}
            indices=[dic_ids[e] for e in ids_]
            embeds=[ embeds[i] for i in indices]
        if embeddings_source == "T5":
            embeds = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_embeds.npy")
            ids = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
            dic_ids={e:i for i, e in enumerate(ids)}
            indices=[dic_ids[e] for e in ids_]
            embeds=[ embeds[i] for i in indices]

        if datatype=="train":
            df_labels=pd.read_pickle(PATH_LABELS)
            
            return embeds,ids,df_labels.loc[ids]
        return embeds,ids
        
 

# 6. Pytorch Models Architectures

In [ ]:
import tensorflow as tf

class MultiLayerPerceptron_v0(tf.keras.Model):

    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
                
         
        self.linear1 = tf.keras.layers.Dense(256)
        #self.activation1 = tf.keras.layers.ReLU()
        self.linear2 = tf.keras.layers.Dense(128)
        #self.activation2 = tf.keras.layers.ReLU()
        self.linear2 = tf.keras.layers.Dense(56)
        
        self.linear3 = tf.keras.layers.Dense(num_classes,activation="sigmoid")

    def call(self, x):

        x = self.linear1(x)
        #x = self.activation1(x)
        x = self.linear2(x)
        # x = self.activation2(x)
        x = self.linear3(x)
        return x
import tensorflow as tf

class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, units):
        super(ResidualBlock, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units, activation='relu')

    def call(self, x):
        residual = x
        x = self.fc1(x)
        x = self.fc2(x)
        x = x + residual
        return x

class MultiLayerPerceptron(tf.keras.Model):
    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(128, activation='relu')
        self.res_block1 = ResidualBlock(128)
        self.res_block2 = ResidualBlock(128)
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.fc3(x)
        return x




#models_t5_1500_cnn_v0_2d
class CNN1D(tf.keras.Model):  #v0_2d
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        # (batch_size, channels, embed_size)
        self.conv1 = tf.keras.layers.Conv1D(filters=3, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 3, embed_size)
        self.pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        # (batch_size, 3, embed_size/2 = 512)
        self.conv2 = tf.keras.layers.Conv1D(filters=3, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 8, embed_size/2 = 512)
        self.pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        
        self.flatten=tf.keras.layers.Flatten()
        # (batch_size, 8, embed_size/4 = 256)
        self.fc1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units=num_classes,activation="sigmoid")

    def call(self, x):
        #x = tf.reshape(x, [-1, tf.shape(x)[1],1])
        x=self.conv1(x)
        x = self.pool1(x)
        x = self.pool2(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x





# 7. Train the Model

In [ ]:
import pandas as pd

X = np.load(os.path.join(PATH_DATAFRAMES_2D,f"embeds_{embeddings_source1}-{embeddings_source2}.npy"))
y = pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   config. num_labels}.pkl"))
#X=np.concatenate([X_train,X_validate])
#y=pd.concat([y_train,y_validate])

num_classes=y.shape[1]
print(f" Num classes {num_classes}")

In [ ]:
display(y.head())

In [ ]:
w_use=weights.loc[y.columns].values
w_use.shape

In [ ]:
labels = pd.read_csv(config.train_labels_path, sep = "\t")
terms=labels[["term","aspect"]].drop_duplicates().set_index("term").loc[y.columns]
aspects={}
for aspect,df_terms in terms.groupby("aspect"):
    terms_index=list(df_terms.index.values)
    display(df_terms.head(5))
    indices = list(np.where(np.isin(y.columns,    terms_index))[0])
    aspects[aspect]=indices
terms.shape

In [ ]:
for u in aspects.values():
    print(len(u))

In [ ]:
from sklearn.metrics import f1_score

def compute_metrics_(tau_arr, g, pred, toi, n_gt, wn_gt=None, ic_arr=None):

    metrics = np.zeros((len(tau_arr), 7), dtype='float')  # cov, pr, rc, wpr, wrc, ru, mi

    for i, tau in enumerate(tau_arr):

        p = solidify_prediction(pred[:, toi], tau)

        # number of proteins with at least one term predicted with score >= tau
        metrics[i, 0] = (p.sum(axis=1) > 0).sum()

        # Terms subsets
        intersection = np.logical_and(p, g)  # TP

        

        # Subsets size
        n_pred = p.sum(axis=1)
        n_intersection = intersection.sum(axis=1)

        # Precision, recall
        metrics[i, 1] = np.divide(n_intersection, n_pred, out=np.zeros_like(n_intersection, dtype='float'),
                                  where=n_pred > 0).sum() 
        metrics[i, 2] = np.divide(n_intersection, n_gt, out=np.zeros_like(n_gt, dtype='float'), where=n_gt > 0).sum()

        if ic_arr is not None:
            # Terms subsets
            remaining = np.logical_and(np.logical_not(p), g)  # FN --> not predicted but in the ground truth
            mis = np.logical_and(p, np.logical_not(g))  # FP --> predicted but not in the ground truth

            # Weighted precision, recall
            #wn_pred = (p * ic_arr[toi]).sum(axis=1)
            wn_pred=np.dot(p , ic_arr[toi])
            #wn_intersection = (intersection * ic_arr[toi]).sum(axis=1)
            wn_intersection = np.dot(intersection ,ic_arr[toi])
            
            #print(f"Shape {wn_intersection.shape},{wn_pred.shape},  {n_intersection.shape},{n_pred.shape}")
            metrics[i, 3] = np.divide(wn_intersection.reshape(-1), wn_pred.reshape(-1), out=np.zeros_like(n_intersection, dtype='float'),
                                      where=wn_pred.reshape(-1) > 0).sum() 
            metrics[i, 4] = np.divide(wn_intersection.reshape(-1), wn_gt.reshape(-1), out=np.zeros_like(n_intersection, dtype='float'),
                                      where=wn_gt.reshape(-1) > 0).sum() 

            # Misinformation, remaining uncertainty
            #metrics[i, 5] = (remaining * ic_arr[toi]).sum(axis=1).sum()
            metrics[i, 5] = np.dot(remaining , ic_arr[toi]).sum()
            #metrics[i, 6] = (mis * ic_arr[toi]).sum(axis=1).sum()
            metrics[i, 6] = np.dot(mis , ic_arr[toi]).sum()
    return metrics
# computes the f metric for each precision and recall in the input arrays
def compute_f(pr, rc):
    n = 2 * pr * rc
    d = pr + rc
    return np.divide(n, d, out=np.zeros_like(n, dtype=float), where=d != 0)


def compute_s(ru, mi):
    return np.sqrt(ru**2 + mi**2)
    # return np.where(np.isnan(ru), mi, np.sqrt(ru + np.nan_to_num(mi)))

def compute_metrics(pred, gt, toi, tau_arr, ic_arr=None, n_cpu=0):
    """
    Takes the prediction and the ground truth and for each threshold in tau_arr
    calculates the confusion matrix and returns the coverage,
    precision, recall, remaining uncertainty and misinformation.
    Toi is the list of terms (indexes) to be considered
    """
    g = gt[:, toi]
    n_gt = g.sum(axis=1)
    wn_gt = None
    if ic_arr is not None:
        #wn_gt = (g * ic_arr[toi]).sum(axis=1)
        wn_gt =np.dot(g,ic_arr[toi])

    
    
    
    arg_lists = [[tau_arr, g, pred, toi, n_gt, wn_gt, ic_arr] for tau_arr in np.array_split(tau_arr, 1)]
    #print(f" Nº  de cpus {n_cpu}, nº de args {len(   arg_lists)}")
    #with mp.Pool(processes=n_cpu) as pool:
    #    metrics = np.concatenate(pool.starmap(compute_metrics_, arg_lists), axis=0)

    results = []
    for args in arg_lists:
        result = compute_metrics_(*args)
        results.append(result)

    metrics = np.concatenate(results, axis=0)
    metrics=pd.DataFrame(metrics, columns=["cov", "pr", "rc", "wpr", "wrc", "ru", "mi"])
    for column in ["pr", "rc", "wpr", "wrc", "ru", "mi"]:
        metrics[column] = np.divide(metrics[column], metrics["cov"], out=np.zeros_like(metrics[column], dtype='float'), where=metrics["cov"] > 0)

    ne = np.full(len(tau_arr), gt.shape[0])
    metrics['ns'] = [""] * len(tau_arr)
    metrics['tau'] = tau_arr
    metrics['cov'] = np.divide(metrics['cov'], ne, out=np.zeros_like(metrics['cov'], dtype='float'), where=ne > 0)
    metrics['f'] = compute_f(metrics['pr'], metrics['rc'])
    metrics['wf'] = compute_f(metrics['wpr'], metrics['wrc'])
    metrics['s'] = compute_s(metrics['ru'], metrics['mi'])

    return metrics

# Return a mask for all the predictions (matrix) >= tau
def solidify_prediction(pred, tau):
    return pred >= tau
def custom_f1_score(y_validate,y_pred):
            max_scores={}
            max_umbrales={}
            scores_dict={}
            for aspect,indice in aspects.items():
                scores_dict[aspect]={}
                max_score=0
                max_umbral=0
                for umbral in np.arange(0.05,0.5,0.05):

                    y_pred_=(y_pred>umbral).astype(int)[:,indice]
                    scores_sums=y_pred_.sum(axis=1)
            
                    n=(scores_sums>0.5).sum()
                    #print(f"n = {n} Secuencias con algun score 1 {n/len(y_pred_)}")
                    y_validate_=y_validate.values[:,indice]

                    y_validate_sums=y_validate_.sum(axis=1)
                    w_use_=w_use[indice,:]

                    f1_scores = f1_score(y_validate_, y_pred_, average=None)
                    
                    t_p=y_pred_*y_validate_
                    
                    numerator=(np.dot(t_p,w_use_))
                    precision_denom=np.dot(y_pred_,w_use_)
                    precision_denom[precision_denom==0]=1
                    weighted_precision= np.sum(   numerator /  precision_denom) / n


                    
                    recall_denom= np.dot(y_validate_,w_use_)
                    recall_denom[recall_denom==0]=1

                
                    weighted_recall=    np.sum(numerator /   recall_denom) / len(y_pred_)
                    average_f1_score = 2*weighted_precision* weighted_recall/( weighted_recall+ weighted_precision)
            
                    scores_dict[aspect][umbral]= average_f1_score 
                # Calcula el F1-score promedio
                #average_f1_score = np.mean(f1_scores)
                    if  average_f1_score > max_score:
                        max_score=average_f1_score 
                        max_umbral=umbral
                max_scores[aspect]= max_score
                max_umbrales[aspect]=umbral
            
            max_score=np.mean(list(max_scores.values()))
            print(f"Scores {scores_dict}")
            # Print the F1 score
            print('F1 Score:', max_score)

class PredictionFnCallback(tf.keras.callbacks.Callback):
    
    def __init__(self,X_validate, y_validate, model=None, verbose=0):
        
               if not model is None: self.model = model
               self.verbose = verbose
               self.X_validate = X_validate
               self.y_validate = y_validate
               self.num_classes=num_classes
            
    def on_epoch_end(self, epoch, logs=None):
       
        y_pred=(self.model.predict(self.X_validate))
        #custom_f1_score(y_validate,y_pred)
        ms=[]
        for aspect,indice in aspects.items():
            ms_=(compute_metrics(y_pred, self.y_validate.values, indice, np.arange(0.1,0.5,0.05), ic_arr=w_use, n_cpu=0))
            ms.append(ms_.wf.max())
        print(np.mean(ms))
            

def custom_f1_score(y_validate,y_pred):
        #custom_f1_score(y_validate,y_pred)
        ms=[]
        for aspect,indice in aspects.items():
            ms_=(compute_metrics(y_pred, y_validate.values, indice, np.arange(0.1,0.5,0.05), ic_arr=w_use, n_cpu=0))
            ms.append(ms_.wf.max())
        print(np.mean(ms))


In [ ]:
import tensorflow as tf

from sklearn.model_selection import KFold
n_splits = 5

# Crear una instancia de KFold
kfold = KFold(n_splits=n_splits)


In [ ]:
models = []

EPOCH_LOAD="10"
VALIDATE = True
predictions_t=None
if TRAIN:
    for fold,(train_indice, test_indice) in enumerate(kfold.split(range(len(X)))):
    
        checkpoint_path = PATHS["CHECKPOINT"][config.env].format(fold) + '/modelo-{epoch:02d}'
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=False,
            save_freq='epoch',

        )
        if model_name=="cn":
            model = CNN1D( num_classes)
        elif model_name=="bp":
            model =MultiLayerPerceptron(num_classes)
        optimizer = tf.keras.optimizers.Adam()
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

        model.compile(optimizer=optimizer,loss=loss_fn,metrics=['binary_accuracy', tf.keras.metrics.AUC()])
        model.fit(x=X[train_indice],y=y.iloc[train_indice],epochs=10,batch_size=32, callbacks=[PredictionFnCallback(X[test_indice],y.iloc[test_indice]),checkpoint_callback],)
        models.append(model)
        if VALIDATE:
            loaded_model = tf.keras.models.load_model( PATHS["CHECKPOINT"][config.env].format(fold)+f'/modelo-{EPOCH_LOAD}')
            # Use the loaded model for predictions or further operations
            predictions = loaded_model.predict(X[test_indice])

            custom_f1_score(y.iloc[test_indice],np.round(predictions,3) )

            print(np.round(predictions,3)[:10])
            predictions=np.round(predictions,3)
            predictions=pd.DataFrame(predictions,index=y.iloc[test_indice].index,columns=y.iloc[test_indice].columns,)
            predictions=predictions.stack()
            predictions=predictions[predictions>0]
            print(predictions.shape)
            predictions_t=pd.concat([predictions_t,predictions])
            predictions.to_csv(PATHS["PREDICTIONS"][config.env].format(fold),header=False, index=True, sep="\t")
            labels.loc[labels.EntryID.isin(y.iloc[test_indice].index)].to_csv(PATHS["LABELS"][config.env].format(fold),sep="\t",index=False)

In [ ]:
def get_dataset( datatype, embeddings_source):
        ids_= np.load(f"{PATH}" + embeds_map["T5"] + "/" + datatype + "_ids.npy")
        if embeddings_source in ["ProtBERT", "EMS2"]:
            embeds = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_embeddings.npy")
            ids = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
            dic_ids={e:i for i,e in enumerate(ids)}
            indices=[dic_ids[e] for e in ids_]
            embeds=np.array([embeds[i] for i in indices])
        
        if embeddings_source == "T5":
            embeds = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_embeds.npy")
            ids = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
            

        if datatype=="train":
            df_labels=pd.read_pickle(PATH_LABELS)
            
            return embeds,ids,df_labels.loc[ids]
        return embeds,ids_
        

In [ ]:
X_test1, ids_test= get_dataset( "test", embeddings_source1)

X_test2, _= get_dataset( "test", embeddings_source2)
if X_test2.shape[1]>  X_test1.shape[1]:
        X_test1 = np.pad(X_test1,((0,0),(0,X_test2.shape[1]-X_test1.shape[1])))
X_test=np.concatenate([np.expand_dims(X_test1,-1),np.expand_dims(X_test2,-1)],axis=-1)



predictions_test_df_t =None
for fold in range(n_splits):
    loaded_model = tf.keras.models.load_model(f'models_{fold}/modelo-10')

    predictions_test = np.round(loaded_model.predict(X_test),3)
    print(predictions_test.shape)
    print(predictions_test[:10])



    print(len(ids_test),len(y.columns))
    predictions_test_df=pd.DataFrame(predictions_test,index=ids_test,columns=y.columns,)
    print((predictions_test_df==0).sum().max())




    predictions_test_df=predictions_test_df.stack()
    
    if predictions_test_df_t is None:
        predictions_test_df_t= predictions_test_df
    else:
        predictions_test_df_t=     predictions_test_df_t+ predictions_test_df
        
        
            
predictions_test_df_t=predictions_test_df_t/n_splits
predictions_test_df=predictions_test_df_t
print(predictions_test_df.shape)


predictions_test_df=predictions_test_df[predictions_test_df>0]
print(predictions_test_df.shape)

predictions_test_df.to_csv("submission.tsv",header=False, index=True, sep="\t")


df_submission = pd.read_csv("submission.tsv",header=None, sep="\t")
display(df_submission)
                            

In [ ]:
predictions_test_df